In [ ]:
import heapq


MONKEY_AT_BOX = 1 << 0
BOX_AT_BANANA = 1 << 1
MONKEY_ON_BOX = 1 << 2
HAS_BANANA   = 1 << 3

GOAL_STATE = HAS_BANANA


def is_goal_state(state):
    return state & GOAL_STATE

def get_next_states(state):
    actions = []

    if not (state & MONKEY_AT_BOX):
        actions.append((state | MONKEY_AT_BOX, "Walk to the box"))

    if (state & MONKEY_AT_BOX) and not (state & BOX_AT_BANANA):
        actions.append((state | BOX_AT_BANANA, "Push the box to the bananas"))


    if (state & MONKEY_AT_BOX) and not (state & MONKEY_ON_BOX):
        actions.append((state | MONKEY_ON_BOX, "Climb onto the box"))


    if (state & MONKEY_ON_BOX) and (state & BOX_AT_BANANA) and not (state & HAS_BANANA):
        actions.append((state | HAS_BANANA, "Grab the banana"))

    return actions


heuristics = {}
for s in range(16):
    if s & HAS_BANANA:
        heuristics[s] = 0
    else:
        h = 0
        if not (s & BOX_AT_BANANA): h += 1
        if not (s & MONKEY_ON_BOX): h += 1
        if not (s & HAS_BANANA):    h += 1
        heuristics[s] = h

def heuristic(state):
    return heuristics[state]


def a_star_search(start_state):
    open_list = [(heuristic(start_state), 0, start_state)]
    g_scores = {start_state: 0}
    parent = {start_state: (None, None)}  # state -> (prev_state, action)
    visited = set()

    while open_list:
        f, g, state = heapq.heappop(open_list)

        if is_goal_state(state):
            return reconstruct_path(parent, state, g)

        if state in visited:
            continue
        visited.add(state)

        for next_state, action in get_next_states(state):
            new_g = g + 1
            if new_g < g_scores.get(next_state, float("inf")):
                g_scores[next_state] = new_g
                parent[next_state] = (state, action)
                heapq.heappush(open_list, (new_g + heuristic(next_state), new_g, next_state))

    return ["No solution found."]


def reconstruct_path(parent, state, cost):
    path = []
    while parent[state][0] is not None:
        state, action = parent[state]
        path.append(action)
    return path[::-1] + [f"Goal reached! The banana is ours! Final Cost: {cost}"]

if __name__ == "__main__":
    initial_state = 0b0000  # monkey not at box, box not at banana, monkey not on box, no banana
    print("Optimized A* for the Monkey and Banana problem.\n")
    for step in a_star_search(initial_state):
        print(step)

